In [47]:
import pandas as pd
from geo import geocode
import importlib
import plotly.express as px

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
importlib.reload(geocode)
importlib.reload(px)

<module 'plotly.express' from '/Users/DanPfeiffer/Documents/code/data-collector/env/lib/python3.11/site-packages/plotly/express/__init__.py'>

In [3]:
box_scores_df = pd.read_csv("../../output/box_scores/box_scores_df.csv")

In [4]:
#box_report = ProfileReport(box_scores_df,
#                           explorative=True)

In [5]:
# box_report.to_file("../output/box_scores/box_score_report.html")

In [7]:
box_scores_df.columns

Index(['id', 'uid', 'date', 'attendance', 'time_valid', 'neutral_site',
       'conference_competition', 'play_by_play_available', 'recent',
       'start_date', 'notes_type', 'notes_headline', 'broadcast_market',
       'broadcast_name', 'type_id', 'type_abbreviation', 'venue_id',
       'venue_full_name', 'venue_address_city', 'venue_address_state',
       'venue_indoor', 'status_clock', 'status_display_clock', 'status_period',
       'status_type_id', 'status_type_name', 'status_type_state',
       'status_type_completed', 'status_type_description',
       'status_type_detail', 'status_type_short_detail', 'status_is_tbd_flex',
       'format_regulation_periods', 'home_id', 'home_uid', 'home_location',
       'home_name', 'home_abbreviation', 'home_display_name',
       'home_short_display_name', 'home_color', 'home_alternate_color',
       'home_is_active', 'home_venue_id', 'home_logo', 'home_score',
       'home_winner', 'home_current_rank', 'home_linescores', 'home_records',
     

In [41]:
cols_to_keep = ['id', 'uid', 'date', 'attendance', 'neutral_site',
       'start_date', 'notes_type', 'venue_id',
       'venue_full_name', 'venue_address_city', 'venue_address_state',
       'venue_indoor',      'home_id', 'home_uid', 'home_location',
       'home_name', 'home_abbreviation', 'home_display_name',
       'home_short_display_name',
       'home_is_active', 'home_venue_id', 'home_score',
       'home_winner',       
       'away_id', 'away_uid', 'away_location', 'away_name',
       'away_abbreviation', 'away_display_name', 'away_short_display_name',
       'away_is_active', 'away_venue_id',
       'away_logo', 'away_score', 'away_winner', 'game_id', 'season', 'season_type',
       'week', 'status_type_alt_detail']

In [42]:
box_scores_df = box_scores_df[cols_to_keep]

In [27]:
stadium_df = box_scores_df[[
    "venue_id", 
    "venue_full_name", 
    "venue_address_city",
    "venue_address_state",
    "attendance"
]].groupby([
    "venue_id", 
    "venue_full_name",
    "venue_address_city",
    "venue_address_state"
]).mean().reset_index().sort_values(by=[
    "venue_address_state", 
    "venue_address_city"
])

In [32]:
stadium_df = geocode.geocode_dataframe(df=stadium_df, 
                                       city_colname="venue_address_city", 
                                       state_colname="venue_address_state")

In [34]:
stadium_df.to_csv("reference/stadiums.csv", index=False)

In [60]:
box_scores_df["score_differential"] = box_scores_df["home_score"] - box_scores_df["away_score"]
box_scores_df["abs_score_differential"] = abs(box_scores_df["score_differential"])
box_scores_df["total_score"] = box_scores_df["home_score"] + box_scores_df["away_score"]
box_scores_df["score_differential_percentile"] = box_scores_df['score_differential'].rank(pct=True)
box_scores_df["total_score_percentile"] = box_scores_df['total_score'].rank(pct=True)

In [52]:
score_cols = ["home_score",
    "away_score",
    "score_differential",
    "abs_score_differential",
    "total_score"]
box_scores_df[score_cols].describe()

,home_score,away_score,score_differential,abs_score_differential,total_score
count,6265.000000,6265.000000,6265.000000,6265.000000,6265.000000
mean,22.821868,20.792338,2.029529,11.264645,43.614206
std,10.188918,9.877578,14.313994,9.060771,14.066655
min,0.000000,0.000000,-49.000000,0.000000,0.000000
25%,16.000000,14.000000,-7.000000,4.000000,34.000000
50%,23.000000,20.000000,3.000000,8.000000,43.000000
75%,30.000000,27.000000,10.000000,17.000000,52.000000
max,70.000000,59.000000,59.000000,59.000000,105.000000


In [54]:
fig = px.histogram(box_scores_df, x=["home_score", "away_score"], histnorm="percent")
fig.write_html("../output/visualizations/score_histogram.html")

In [55]:
for col in score_cols:
    title = f"{col}_histogram"
    fig = px.histogram(box_scores_df, x=col, title=title, histnorm="percent")
    fig.write_html(f"../output/visualizations/{title}.html")
    for y_col in score_cols:
        if y_col != col:
            title = f"{col}_vs_{y_col}_scatter"
            fig = px.scatter(box_scores_df, x=col, y=y_col, title=title)
            fig.write_html(f"../output/visualizations/{title}.html")


In [59]:
title = f"game_line_scatter"
fig=px.scatter(box_scores_df, x="score_differential", y="score_differential_percentile",title=title)
fig.write_html(f"../output/visualizations/{title}.html")

In [61]:
title = f"total_line_scatter"
fig=px.scatter(box_scores_df, x="total_score", y="total_score_percentile",title=title)
fig.write_html(f"../output/visualizations/{title}.html")